# Imports

In [1]:
# Own Packages
from Masterarbeit_utils.model_utils import get_tokenizer, load_and_modify_model, load_pretrained_Tokenizer

# Site-Packages
import dask.dataframe as dd
import torch
import psutil
import os
import sys
import pickle as pk
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, OPTForCausalLM
from tokenizers.processors import TemplateProcessing
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from torch.utils.data import Dataset
sys.version, sys.executable

/home/worker/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


('3.11.2 (main, May 30 2023, 17:45:26) [GCC 12.2.0]', '/usr/bin/python3')

# Parameters

In [2]:
"""
The Paths to important folders have to be changed for your system.
"""

# This folder will be created and filled with txt.files for each sample after you run the Pytorch Dataset Notebook
dataset_folder = f'data/dataset_samples'

# The folder at which the model will be saved. This folder has to be created for your system 
model_folder = f'data/models/gal_125_1'
os.makedirs(model_folder, exist_ok=True)

# The folder at which the training progress will be logged
log_folder = 'data/models/gal_125_1/logs'
os.makedirs(log_folder, exist_ok=True)

# Folder at which all pickle files are stored. This folder is fixed for this project and should not be changed
dump_dir = r'PK_DUMP'

# Model parameters 
'''
mini	125 M
base	1.3 B
standard	6.7 B
large	30 B
huge	120 B'''
base_model_name = 'mini'

# All new Torch-objects will be by default in this dtype
# if default_type = float16 fp16 must be False
default_dtype = torch.bfloat16
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_default_dtype(default_dtype)

# Default device on which the model will be loaded
default_device = 'cuda:0'

# Number of GPUs the model will be parallelised to 
num_gpus = 1
# If you change 'default_device' to 'cpu', make sure to set num_gpus to zero.
if default_device == 'cpu':
    num_gpus = 0

tensor_parallel = False
n_f_terms = None # Will be calculated

# Training parameters!
output_dir = model_folder
num_train_epochs = 1
per_device_train_batch_size = 10
per_device_eval_batch_size = per_device_train_batch_size
save_strategy = "epoch"
logging_strategy = "steps"
evaluation_strategy = "steps"
eval_steps = 1000000 # eval_steps = number of samples until evaluation is performed.
eval_steps = int(eval_steps/per_device_train_batch_size)
learning_rate = 5e-5
weight_decay = 0.0
seed = 42

# This that could improve performance
dataloader_num_workers = 4
# sytem varables that must be set for the tokenizer
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
torch_compile = False
# V-Ram reduction only if default_dtype= float32
fp16=False
if default_dtype == torch.float16:
    fp16=False
bf16=False
tf32=True

# Creating the Tokenizer

In [3]:
# Loads a pretrained Tokenizer for the galactica model and adds an additional token for each F-Term
tokenizer = get_tokenizer(dump_dir)

# The Tokenizer contained initially 50000 Tokens which are stored as the vocab-size.
# The vocab_size attribute is not updated when the additional tokens are added to the tokenizer
n_f_terms = len(tokenizer) - tokenizer.vocab_size
print(f'There are {n_f_terms} different F-Terms in the Tokenizer vocabulary!')

There are 379753 different F-Terms in the Tokenizer vocabulary!


# Creating the dataset

In [4]:
# Samples in train 6385601
# Samples in val 1596401

class JapPatDataset(Dataset):
    """Dataset containing Japanese patents and their F-Term classification"""
    def __init__(self, data_folder, tokenizer):
        """
        data_folder: path to folder containing the text samples
        tokenizer: tokenizer instance with added additional Tokens for F-Terms
        """
        super(Dataset).__init__()
        self.data_folder = data_folder
        # This has to be manually set to the ammount of files in the 'dataset_samples' folder. Calculating the number of files in this folder would take forever.
        # A to low number would lead to samples missing from the dataset.
        # A to high number would raise a FileNotFound error.
        self.l = len(os.listdir(data_folder))
        self.tokenizer = tokenizer
        
    def __len__(self):
        return self.l
    
    def __getitem__(self, idx):
        try:
            with open(f'{self.data_folder}/{idx}.txt', 'r', encoding='utf-8') as f:
                item = f.read()
        except FileNotFoundError:
            raise FileNotFoundError
        
        # Tokenizing the item 
        # The Tokenizer will return a dict with the encoded text as 'input_ids', 
        # a mask which shows the tokens types this will not be needed for our applications
        # and a mask for the attention mechanism as 'attention_mask' The attention mask will be needed to indicate, that the 
        # model should not attend to <pad> tokens.
        return self.tokenizer(item)  

In [5]:
train_dataset = JapPatDataset(f'{dataset_folder}/train', tokenizer)
validation_dataset = JapPatDataset(f'{dataset_folder}/validation', tokenizer)

In [6]:
# The pretrained model is loaded from Huggingface.
# The token-embedding is expanded for all f-terms and the output embeddings is compleatly replaced by a F-Term classification head.
model = load_and_modify_model(base_model_name, default_dtype, tensor_parallel, num_gpus, n_f_terms, default_device)
print(f'The model interprets token-index {model.config.bos_token_id} as the beginning of a sequence and {model.config.eos_token_id} as the end')

cuda:0
The model interprets token-index 0 as the beginning of a sequence and 2 as the end


# Creating the Trainer Class by Subclassing from Huggingface-Trainer

In [7]:
# Subclassing the Huggingface Trainer class to use custome code to calculate the loss
# The labels used for the loss are generated and the labels for the text tokens are set to -100 to ignore their loss,
# because the modified model can't predict text-tokens
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs: bool=False):
        """
        model: model which should be trained.
        inputs: A padded batch of samples from the dataset.
        return_outputs: Indicates if the whole output of the model is returned or not.
        """
        # Removing the token_type_ids because we don't need them
        try:
            inputs.pop('token_type_ids')
        except KeyError:
            pass
        labels = inputs['input_ids'].clone()
        # Generating the labels, because the model can only predict F-Terms but also can interpret Text-Tokens as input, 
        # The maximum token idx is 50000 higher than the maximum output_idx
        labels = labels - 50000
        # All text tokens have token_idx below 50000 after substracting 50000 they are negative and 
        # are now set to -100 to ignore them when the loss is computed
        labels[labels<0] = -100
        # generating the output of the model
        # It is a dict of 'loss', 'logits' and 'past_key_values'
        outputs = model(**inputs, output_attentions=False, output_hidden_states=False, return_dict=True, labels=labels)
        loss = outputs['loss']
        #print(loss, outputs['logits'].shape, outputs['logits'][0, 0, :100])
        print('loss:',loss.item(), end='\r')
        return (loss, outputs) if return_outputs else loss
        

# Finding the Optimal Batch-Size

# Extracting the number of tokens per sample

file_indices = np.arange(train_dataset.l)
empty_df = pd.DataFrame(file_indices, columns=['Files'])
lengths_df = dd.from_pandas(empty_df, chunksize=100000)

def load_file(index):
    with open(f'{train_dataset.data_folder}/{index}.txt', 'r', encoding='utf-8') as f:
        txt = f.read()
    tokens = train_dataset.tokenizer(txt)
    return len(tokens['input_ids'])

lengths_df['Lengths'] = lengths_df['Files'].map(load_file)

# Finding the longest sample
n_tokens = 0
max_l = 0
for i, l in enumerate(lengths_df['Lengths']):
    n_tokens += l
    if l > max_l:
        max_l = l
    if i%1000 == 0:
        print(f'Processed {i} samples, maximum found lengths = {max_l}, number of total tokens = {n_tokens}', end= '\r')

In [8]:
class DummyDataset(Dataset):
    def __init__(self, l):
        self.l = 100
        self.sample = {'input_ids': torch.tensor([int(x) for x in np.random.randint(50000, tokenizer.vocab_size + 50000, size=l)]),  'attention_mask': torch.tensor([1 for _ in range(l)])}

    def __len__(self):
        return self.l

    def __getitem__(self, idx):
        return self.sample
        

In [9]:
max_l = 469

batch_size = 12
print(f'Testing batch_size {batch_size}', end='\r')
torch.cuda.empty_cache()
    
# The TrainingArguments class is a class which stores multiple parameters for the Custom-trainer of the model.

training_args = TrainingArguments(
    output_dir=output_dir,          
    logging_dir=log_folder,
    num_train_epochs=num_train_epochs,             
    per_device_train_batch_size=per_device_train_batch_size,    
    per_device_eval_batch_size=per_device_eval_batch_size,
    save_strategy=save_strategy,
    logging_strategy=logging_strategy,
    evaluation_strategy=evaluation_strategy,
    eval_steps=eval_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    seed=seed,
    dataloader_num_workers=dataloader_num_workers, 
    fp16=fp16,
    bf16=bf16,
    tf32=tf32,
    torch_compile=torch_compile
)

dummy_ds = DummyDataset(max_l)

trainer = CustomTrainer(model=model, args=training_args, train_dataset=dummy_ds, eval_dataset=dummy_ds, data_collator=DataCollatorWithPadding(tokenizer, return_tensors='pt'))


trainer.train()

        


In [10]:
batch_size = 15

for n_tokens in range(300, 1000, 10):
    print(f'Maximum number of tokens per sample for batchsize {batch_size} = {n_tokens}')

    max_l = n_tokens

    torch.cuda.empty_cache()
    dummy_ds = DummyDataset(max_l)
    training_args = TrainingArguments(
            output_dir=output_dir,          # output directory
            num_train_epochs=1,              # total # of training epochs
            per_device_train_batch_size=batch_size,    # batch size per device during training
            save_strategy=save_strategy,
            logging_strategy=logging_strategy,
            evaluation_strategy=evaluation_strategy,
            learning_rate=learning_rate,
            weight_decay=weight_decay)
    
    trainer = CustomTrainer(model=model, args=training_args, train_dataset=dummy_ds, eval_dataset=dummy_ds, data_collator=DataCollatorWithPadding(tokenizer, return_tensors='pt'))
    
    trainer.train()

/home/worker/.local/lib/python3.11/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Maximum number of tokens per sample for batchsize 15 = 300


Step,Training Loss,Validation Loss


Maximum number of tokens per sample for batchsize 15 = 310


/home/worker/.local/lib/python3.11/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Step,Training Loss,Validation Loss


Maximum number of tokens per sample for batchsize 15 = 380


/home/worker/.local/lib/python3.11/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/worker/.local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3508 in        │
│ run_code                                                                                         │
│                                                                                                  │
│   3505 │   │   │   │   if async_:                                                                │
│   3506 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3507 │   │   │   │   else:                                                                     │
│ ❱ 3508 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3509 │   │   │   finally:                                                                      │
│   3510 │   │   │   │   # Reset our crash handler in place                                        │
│   3511 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ in <module>                                                                                      │
│                                                                                                  │
│   19 │                                                                                           │
│   20 │   trainer = CustomTrainer(model=model, args=training_args, train_dataset=dummy_ds, eva    │
│   21 │                                                                                           │
│ ❱ 22 │   trainer.train()                                                                         │
│   23                                                                                             │
│                                                                                                  │
│ /home/worker/.local/lib/python3.11/site-packages/transformers/trainer.py:1664 in train           │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args=args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1667 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /home/worker/.local/lib/python3.11/site-packages/transformers/trainer.py:1940 in                 │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1937 │   │   │   │   │   with model.no_sync():                                                 │
│   1938 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1939 │   │   │   │   else:                                                                     │
│ ❱ 1940 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1941 │   │   │   │                                                                             │
│   1942 │   │   │   │   if (                                                                      │
│   1943 │   │   │   │   │   args.logging_nan_inf_filter     

max_bs_for_1279 = 3\
maximum tokens per sample for batch size 4 = 1034\
maximum tokens per sample for batch size 5 = 864\
maximum tokens per sample for batch size 6 = 740\
maximum tokens per sample for batch size 7 = 650\
maximum tokens per sample for batch size 8 = 570\
maximum tokens per sample for batch size 9 = 520\
maximum tokens per sample for batch size 10 = 470


In [ ]:
import transformers
print(transformers.__version__)
print(python.__version__)

In [ ]:
sys.path

In [ ]:
torch.cuda.empty_cache()

In [ ]:
log_folder = 'data/models/gal_125_1/logs'
os.makedirs(log_folder, exist_ok=True)

# Folder at which all pickle files are stored. This folder is fixed for this project and should not be changed
dump_dir = r'PK_DUMP'

# Model parameters 
'''
mini	125 M
base	1.3 B
standard	6.7 B
large	30 B
huge	120 B'''
base_model_name = 'mini'

# All new Torch-objects will be by default in this dtype
default_dtype = torch.float16
torch.set_default_dtype(default_dtype)

# Default device on which the model will be loaded
default_device = 'cuda:0'

# Number of GPUs the model will be parallelised to 
num_gpus = 1
# If you change 'default_device' to 'cpu', make sure to set num_gpus to zero.
if default_device == 'cpu':
    num_gpus = 0

tensor_parallel = False
n_f_terms = None # Will be calculated

# Training parameters!
output_dir=model_folder
num_train_epochs=3
per_device_train_batch_size=10
per_device_eval_batch_size=10
save_strategy="epoch"
logging_strategy="steps"
evaluation_strategy="steps"
eval_steps = 1000000 # eval_steps = number of samples until evaluation is performed.
eval_steps = int(eval_steps/per_device_train_batch_size)
learning_rate=5e-5
weight_decay=0.0
seed = 42

# This that could improve performance
dataloader_num_workers = 4
torch_compile = True
# V-Ram reduction
fp16=True